In [4]:
# -*- coding: utf-8 -*-
"""
Created on Tue Sep 10 09:50:45 2019
@author: s-moh
"""
import numpy as np
import pandas as pd
import datetime

# dataBefore = 'C:/Users/s-moh/0-Labwork/Rakuten Project/Dataset/RecSys_Dataset_Before/yoochoose-clicks.dat' #Path to Original Training Dataset "Clicks" File
dataTestBefore = '/home/hainh/coccoc/code/week11/GRU4REC-pytorch/Dataset/raw_data/yoochoose-test.dat' #Path to Original Testing Dataset "Clicks" File
# dataAfter = 'C:/Users/s-moh/0-Labwork/Rakuten Project/Dataset/RecSys_Dataset_After/' #Path to Processed Dataset Folder
dayTime = 86400 #Validation Only one day = 86400 seconds

def removeShortSessions(data):
    #delete sessions of length < 1
    sessionLen = data.groupby('SessionID').size() #group by sessionID and get size of each session
    data = data[np.in1d(data.SessionID, sessionLen[sessionLen > 1].index)]
    return data

#Read Dataset in pandas Dataframe (Ignore Category Column)
train = pd.read_csv(dataTestBefore, sep=',', header=None, usecols=[0,1,2], dtype={0:np.int32, 1:str, 2:np.int64})
# test = pd.read_csv(dataTestBefore, sep=',', header=None, usecols=[0,1,2], dtype={0:np.int32, 1:str, 2:np.int64})
train.columns = ['SessionID', 'Time', 'ItemID'] #Headers of dataframe
# test.columns = ['SessionID', 'Time', 'ItemID'] #Headers of dataframe
train['Time']= train.Time.apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%fZ').timestamp()) #Convert time objects to timestamp
# test['Time'] = test.Time.apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%fZ').timestamp()) #Convert time objects to timestamp

#remove sessions of less than 2 interactions
train = removeShortSessions(train)
#delete records of items which appeared less than 5 times
itemLen = train.groupby('ItemID').size() #groupby itemID and get size of each item
train = train[np.in1d(train.ItemID, itemLen[itemLen > 4].index)]
#remove sessions of less than 2 interactions again
train = removeShortSessions(train)

######################################################################################################3
'''
#Separate Data into Train and Test Splits
timeMax = data.Time.max() #maximum time in all records
sessionMaxTime = data.groupby('SessionID').Time.max() #group by sessionID and get the maximum time of each session
sessionTrain = sessionMaxTime[sessionMaxTime < (timeMax - dayTime)].index #training split is all sessions that ended before the last day
sessionTest  = sessionMaxTime[sessionMaxTime >= (timeMax - dayTime)].index #testing split is all sessions has records in the last day
train = data[np.in1d(data.SessionID, sessionTrain)]
test = data[np.in1d(data.SessionID, sessionTest)]
'''
#Delete records in testing split where items are not in training split
# test = test[np.in1d(test.ItemID, train.ItemID)]
#Delete Sessions in testing split which are less than 2
# test = removeShortSessions(test)

#Convert To CSV
#print('Full Training Set has', len(train), 'Events, ', train.SessionID.nunique(), 'Sessions, and', train.ItemID.nunique(), 'Items\n\n')
#train.to_csv(dataAfter + 'recSys15TrainFull.txt', sep='\t', index=False)
# print('Testing Set has', len(test), 'Events, ', test.SessionID.nunique(), 'Sessions, and', test.ItemID.nunique(), 'Items\n\n')
#test.to_csv(dataAfter + 'recSys15Test.txt', sep=',', index=False)

######################################################################################################3
#Separate Training set into Train and Validation Splits
timeMax = train.Time.max()
sessionMaxTime = train.groupby('SessionID').Time.max()
sessionTrain = sessionMaxTime[sessionMaxTime < (timeMax - dayTime)].index #training split is all sessions that ended before the last 2nd day
sessionValid = sessionMaxTime[sessionMaxTime >= (timeMax - dayTime)].index #validation split is all sessions that ended during the last 2nd day
trainTR = train[np.in1d(train.SessionID, sessionTrain)]
trainVD = train[np.in1d(train.SessionID, sessionValid)]
#Delete records in validation split where items are not in training split
trainVD = trainVD[np.in1d(trainVD.ItemID, trainTR.ItemID)]
#Delete Sessions in testing split which are less than 2
trainVD = removeShortSessions(trainVD)
#Convert To CSV
print('Training Set has', len(trainTR), 'Events, ', trainTR.SessionID.nunique(), 'Sessions, and', trainTR.ItemID.nunique(), 'Items\n\n')
#trainTR.to_csv(dataAfter + 'recSys15TrainOnly.txt', sep=',', index=False)
print('Validation Set has', len(trainVD), 'Events, ', trainVD.SessionID.nunique(), 'Sessions, and', trainVD.ItemID.nunique(), 'Items\n\n')
#trainVD.to_csv(dataAfter + 'recSys15Valid.txt', sep=',', index=False)

Training Set has 7887595 Events,  1986830 Sessions, and 27353 Items


Validation Set has 17707 Events,  3785 Sessions, and 3160 Items




In [5]:
trainTR

,SessionID,Time,ItemID
0,5,1.396866e+09,214530776
1,5,1.396866e+09,214530776
2,5,1.396866e+09,214530776
3,10,1.396572e+09,214820942
4,10,1.396572e+09,214826810
...,...,...,...
8251786,11299820,1.411608e+09,214853094
8251787,11299815,1.411700e+09,214854804
8251788,11299815,1.411700e+09,214714715
8251789,11299810,1.411722e+09,214546123


In [6]:
trainVD

,SessionID,Time,ItemID
8046986,11264965,1.411980e+09,214545928
8046987,11264965,1.411980e+09,214545928
8047046,11264880,1.411992e+09,214516112
8047047,11264880,1.411993e+09,214692868
8047048,11264880,1.411993e+09,214561678
...,...,...,...
8251757,11299785,1.411988e+09,214858792
8251758,11299785,1.411988e+09,214858905
8251759,11299785,1.411988e+09,214857701
8251760,11299785,1.411988e+09,214556563


In [ ]:
# Load the model checkpoint
model = load_model_checkpoint('gru4rec_checkpoint')

# Preprocess the input sequence
preprocessed_sequence = preprocess_input_sequence(input_sequence)

# Forward pass through the model
predictions = model.predict(preprocessed_sequence)

# Interpret the predictions
predicted_item = get_predicted_item(predictions)

# Print the predicted item
print('Predicted item:', predicted_item)


In [3]:
import lib

In [5]:
import os

In [6]:
train_data = lib.Dataset('/home/hainh/coccoc/code/week11/GRU4REC-pytorch/Dataset/RecSys_Dataset_After/recSys15TrainOnly.txt')
# valid_data = lib.Dataset(, itemmap=train_data.itemmap)

#set all the parameters according to the defined arguments
input_size = len(train_data.items)


In [10]:
import torch
import torch.nn as nn
import numpy as np

# Define the GRU4REC model architecture
class GRU4REC(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(GRU4REC, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        _, hidden = self.gru(x)
        output = self.fc(hidden.squeeze(0))
        return output

# Load the model checkpoint
model = GRU4REC(16, 100, 16)
model.load_state_dict(torch.load('/home/hainh/coccoc/code/week11/GRU4REC-pytorch/checkpoint/06072142/model_00004.pt', map_location=torch.device('cpu')))
model.eval()

# Define a function to preprocess the input sequence
def preprocess_input_sequence(input_sequence):
    # Perform any necessary preprocessing on the input sequence
    # For example, you might convert the sequence to a tensor
    preprocessed_sequence = torch.tensor(input_sequence, dtype=torch.float32).unsqueeze(0)
    
    return preprocessed_sequence

# Define a function to interpret the model's predictions
def get_predicted_item(predictions):
    # Extract the predicted item from the model's predictions
    predicted_item = torch.argmax(predictions).item()
    
    return predicted_item

# Define your input sequence
input_sequence = [1, 2, 3, 4, 5]  # Example input sequence

# Preprocess the input sequence
preprocessed_sequence = preprocess_input_sequence(input_sequence)

# Forward pass through the model
with torch.no_grad():
    predictions = model(preprocessed_sequence)

# Interpret the predictions
predicted_item = get_predicted_item(predictions)

# Print the predicted item
print('Predicted item:', predicted_item)


RuntimeError: Error(s) in loading state_dict for GRU4REC:
	Missing key(s) in state_dict: "gru.weight_ih_l0", "gru.weight_hh_l0", "gru.bias_ih_l0", "gru.bias_hh_l0", "fc.weight", "fc.bias". 
	Unexpected key(s) in state_dict: "model", "args", "epoch", "optim", "loss", "recall", "mrr". 

In [11]:
import torch
import torch.nn as nn
import numpy as np

# Define the GRU4REC model architecture
class GRU4REC(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(GRU4REC, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        _, hidden = self.gru(x)
        output = self.fc(hidden.squeeze(0))
        return output

# Load the model checkpoint
model = GRU4REC(16, 100, 16)
model.load_state_dict(torch.load('/home/hainh/coccoc/code/week11/GRU4REC-pytorch/checkpoint/06072142/model_00004.pt',  map_location=torch.device('cpu')))
model.eval()

# Define a function to preprocess the input sequence
def preprocess_input_sequence(input_sequence):
    # Perform any necessary preprocessing on the input sequence
    # For example, you might convert the sequence to a tensor
    preprocessed_sequence = torch.tensor(input_sequence, dtype=torch.float32).unsqueeze(0)
    
    return preprocessed_sequence

# Define a function to interpret the model's predictions
def get_predicted_item(predictions):
    # Extract the predicted item from the model's predictions
    predicted_item = torch.argmax(predictions).item()
    
    return predicted_item

# Define your input sequence
input_sequence = [1, 2, 3, 4, 5]  # Example input sequence

# Preprocess the input sequence
preprocessed_sequence = preprocess_input_sequence(input_sequence)

# Forward pass through the model
with torch.no_grad():
    predictions = model(preprocessed_sequence)

# Interpret the predictions
predicted_item = get_predicted_item(predictions)

# Print the predicted item
print('Predicted item:', predicted_item)


RuntimeError: Error(s) in loading state_dict for GRU4REC:
	Missing key(s) in state_dict: "gru.weight_ih_l0", "gru.weight_hh_l0", "gru.bias_ih_l0", "gru.bias_hh_l0", "fc.weight", "fc.bias". 
	Unexpected key(s) in state_dict: "model", "args", "epoch", "optim", "loss", "recall", "mrr". 